In [4]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
import re

In [5]:

# ---------------- Lectura y guardado de los documentos en un data frame de pandas.


In [6]:
def get_feelings_df(balance_data, lematizacion):
    from gensim.utils import any2unicode as unicode
    from nltk.stem import SnowballStemmer
    from sklearn.utils import resample
    import os, re, sys, pandas, unidecode

    feelings_folder = './entrenamiento de modelos/datos sentimientos'
    feelings_df = pandas.DataFrame(columns=['text', 'lang', 'sentiment'])
    path, subfolders, files_list = list(os.walk(feelings_folder))[0]
    files_list.sort()

    for i in range(len(files_list)):
        sys.stdout.write("\rPreparando df  " + str(round(((i + 1) / (len(files_list))) * 100, 2)) + "%")
        sys.stdout.flush()
        file_name, file_ext = files_list[i].split(".")

        if file_ext == 'csv':
            file_path = path + "/" + file_name + "." + file_ext
            df = pandas.read_csv(file_path, encoding='utf8', dtype=str, engine='python')
            numero_de_archivo = int(file_name.split("_")[0])

            if numero_de_archivo == 1:
                df = df.filter(['airline_sentiment', 'text'])
                df.columns = ['sentiment', 'text']
                df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 2 or numero_de_archivo == 3 or numero_de_archivo == 4:
                df = df.filter(['sentiment', 'text'])
                df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 5 or numero_de_archivo == 6 or numero_de_archivo == 7:
                df = df.filter(['polarity', 'text'])
                df.columns = ['sentiment', 'text']
                if numero_de_archivo in [5, 6]: df['lang'] = 'es'
                if numero_de_archivo == 7: df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 8:
                df = df.filter(['sentiment', 'text'])
                df.columns = ['sentiment', 'text']
                df['lang'] = 'es'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

    del df
    print("")
    feelings_df.dropna()
    feelings_df.drop_duplicates(subset=['text'], inplace=True)
    feelings_df = feelings_df.loc[feelings_df['sentiment'].isin(['0', '1', '2'])]
    feelings_df = feelings_df.loc[feelings_df['lang'].isin(['es', 'en'])]
    feelings_df.reset_index(drop=True, inplace=True)

    for i, row in feelings_df.iterrows():
        sys.stdout.write("\rNormalizando df " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        feelings_df.at[i, 'text'] = (
            re.sub(' +', ' ', re.sub("http\S+", "", re.sub('\s+', ' ', str(feelings_df.at[i, 'text']))))
        ).strip()

    print("")

    if balance_data == 1:
        feelings_df.dropna()
        feelings_df["Sello"] = 0
        for i, row in feelings_df.iterrows():
            #sys.stdout.write(
             #   "\rCreando sellos de balanceamiento " +
             #   str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2))
             #   + "%"
            #)
            #sys.stdout.flush()
            feelings_df.at[i, 'sello'] = str(feelings_df.at[i, 'lang']) + '_' + str(feelings_df.at[i, 'sentiment'])
        print("\nBalanceando df")
        min_len1 = int(min(feelings_df['sello'].value_counts()))
        df_0 = resample(feelings_df[feelings_df.sello == 'en_0'], replace=False, n_samples=min_len1, random_state=1)
        df_1 = resample(feelings_df[feelings_df.sello == 'en_1'], replace=False, n_samples=min_len1, random_state=1)
        df_2 = resample(feelings_df[feelings_df.sello == 'en_2'], replace=False, n_samples=min_len1, random_state=1)
        df_3 = resample(feelings_df[feelings_df.sello == 'es_0'], replace=False, n_samples=min_len1, random_state=1)
        df_4 = resample(feelings_df[feelings_df.sello == 'es_1'], replace=False, n_samples=min_len1, random_state=1)
        df_5 = resample(feelings_df[feelings_df.sello == 'es_2'], replace=False, n_samples=min_len1, random_state=1)
        feelings_df = pandas.concat([df_0, df_1, df_2, df_3, df_4, df_5])
        feelings_df = feelings_df.filter(['text', 'lang', 'sentiment'])
        feelings_df.reset_index(drop=True, inplace=True)

    if lematizacion == 1:
        stemmer_en = SnowballStemmer('english')
        stemmer_es = SnowballStemmer('spanish')
        for i, row in feelings_df.iterrows():
            #sys.stdout.write("\rLematizando df " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%")
            #sys.stdout.flush()
            if feelings_df.at[i, 'text'] is str and feelings_df.at[i, 'lang'] == 'es':
                feelings_df.at[i, 'text'] = stemmer_es.stem(unidecode.unidecode(
                    unicode(feelings_df.at[i, 'text'].lower(), "utf-8"))
                )
            elif feelings_df.at[i, 'text'] is str and feelings_df.at[i, 'lang'] == 'en':
                feelings_df.at[i, 'text'] = stemmer_en.stem(unidecode.unidecode(
                    unicode(feelings_df.at[i, 'text'].lower(), "utf-8"))
                )
        feelings_df.reset_index(drop=True, inplace=True)
        print("")

    print("Df sentiments entregado\n")
    feelings_df.reset_index(drop=True, inplace=True)
    
    data12 = pd.DataFrame(columns=('text', 'lang', 'sentiment','emoji'))
    emoj=0
    tweet_tokenizer = TweetTokenizer()

    for row in feelings_df.iterrows():
        texto = row[1]['text']
        text = re.sub(r'[\b@]\w+\s{1}', '', texto)#Quitar menciones
        words = tweet_tokenizer.tokenize(text)
        for w in words:
            if  w in emoji.UNICODE_EMOJI:
                emoj = 1
                break
            else: emoj=0

        lenguaje = row[1]['lang']
        sentiment = row[1]['sentiment']
        data12.loc[len(data12)]=[text,lenguaje,sentiment,emoj] 
        feelings_df = data12
    
    return feelings_df

In [7]:
df = get_feelings_df(1,1)

Preparando df  100.0%
Normalizando df 100.0%

Balanceando df

Df sentiments entregado



In [8]:
print("\nTweets en el dataframe original: " + str(df.shape[0]))


Tweets en el dataframe original: 15564


In [9]:
df.columns.values

array(['text', 'lang', 'sentiment', 'emoji'], dtype=object)

In [10]:
df1 = df

In [11]:
#----------------- Lexicones
#Lectura de SentiWordNet Obtenido de 
#https://www.nltk.org/_modules/nltk/corpus/reader/sentiwordnet.html

# SentiWordNet[word] = {POS,	ID,	PosScore,	NegScore}
contador = 0
SentiWordNet = dict()
for lines in open('./entrenamiento de modelos/data lexicon/SentiWordNet_3.0.0.txt'):
    if lines.startswith('#'):
        continue
    line = lines.split('\t')
    palabra = line[4].split('#')[0]
    if (palabra in SentiWordNet) or (palabra==''):
        continue
    else:
        SentiWordNet[palabra]={'POS': line[0], 'ID': line[1], 'PosScore': line[2], 'NegScore': line[3]}

In [12]:
# AFFIN[word] = sentiment
AFFIN = dict()
for lines in open('./entrenamiento de modelos/data lexicon/AFFIN-111.txt'):
    AFFIN[lines.split('\t')[0]]=(lines.split('\t')[1]).split('\n')[0]

In [13]:
# Lexicon Esp Pos - Neg[word] = sentiment
ESPpos = list()
ESPneg = list()
for lines in open('./entrenamiento de modelos/data lexicon/negative_words_es.txt'):
    ESPneg.append(lines.split("\n")[0])
for lines1 in open('./entrenamiento de modelos/data lexicon/positive_words_es.txt'):
    ESPpos.append(lines1.split("\n")[0])

In [14]:
tt = TweetTokenizer()
def Lexicon(data1):
    data = data1['text']
    addlex = list()
    i=0
    for frase in data:
        splited = tt.tokenize(frase)
        
        sum_swn_neg = 0
        sum_swn_pos = 0
        affin = 0
        stnet = 0
        contpos = 0
        contneg = 0
        word_stat = 0
        total=0
        for word in splited:
        
            #if word in SentiWordNet.keys():
            #    sum_swn_neg += float(SentiWordNet[word]['NegScore'])
            #    sum_swn_pos += float(SentiWordNet[word]['PosScore'])
            #if word in AFFIN.keys():
            #    affin += float(AFFIN[word])
            if word in ESPpos:
                contpos +=1
            if word in ESPneg:
                contneg +=1
        total = contpos + contneg
        if total!=0:
            total = contpos / total
        else: total=0.5
        #datemo = int(data1['emoji'][i])
        addlex.append([total])
        i+=1
    return addlex

In [15]:
def extension(matriz, data):
    extendLex = np.array(Lexicon(data))
    return np.append(matriz, extendLex, 1)

In [16]:
vectorizer = TfidfVectorizer(max_features=3000, min_df=7, max_df=0.8)

In [17]:
# -------- Español.
df_es = df[df1.lang == 'es']
# -------- Ingles.
df_en = df[df1.lang == 'en']

In [18]:
df_es.shape

(7782, 4)

In [19]:
df_en.shape

(7782, 4)

In [20]:
features = vectorizer.fit_transform(df_es['text']).toarray()

In [21]:
labels = df_es['sentiment']

In [22]:
labels.shape

(7782,)

In [23]:
vectorizacionLexicon = extension(features, df_es)
vectorizacionLexicon.shape

(7782, 2367)

In [24]:
#for i in vectorizacionLexicon: print(i)

In [25]:
data_train, data_test, label_train, label_test = train_test_split(vectorizacionLexicon,labels)

In [26]:
def entrenar_modelos_sentiments_supervisados(modelo_entr,vectorizacionLexicon,labels):
    from nltk.corpus import stopwords
    from sklearn.naive_bayes import GaussianNB,BernoulliNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import classification_report
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    import pandas as pd
    import pickle, os, nltk

    modelos_es = {
        'GBT': AdaBoostClassifier(DecisionTreeClassifier(max_depth=200), n_estimators=10),
        'RF': RandomForestClassifier(n_estimators=100, max_depth=300),
        'NN': MLPClassifier(hidden_layer_sizes=(50, 2), max_iter=100),
        'DT': DecisionTreeClassifier(max_depth=16),
        'NB': GaussianNB()
    }
    
    modelos_en = {
        'GBT': AdaBoostClassifier(DecisionTreeClassifier(max_depth=200), n_estimators=10),
        'RF': RandomForestClassifier(n_estimators=100, max_depth=300),
        'NN': MLPClassifier(hidden_layer_sizes=(50, 2), max_iter=100),
        'DT': DecisionTreeClassifier(max_depth=16),
        'NB': BernoulliNB()
        
    }


    print("")
    nltk.download('stopwords')

    # ---------------- Asignacion de los modelos y vectorizadores.
    # -------- Español.
    modelo_es = modelos_es[modelo_entr]
    #vectorizer_es = TfidfVectorizer(use_idf=True, stop_words=stopwords.words('spanish'))
    # -------- Ingles.
    #modelo_en = modelos_en[modelo_entr]
    #vectorizer_en = TfidfVectorizer(use_idf=True, stop_words=stopwords.words('english'))

    # ---------------- Lectura y separacion de datos.
    #df = pd.DataFrame(get_feelings_df(df_balanceado, df_lematizado))
    # -------- Español.
    #df_es = df[df.lang == 'es']
    # -------- Ingles.
    #df_en = df[df.lang == 'en']
    #del df
    print("Separacion de datos por idioma terminada")

    # ---------------- Separacion en data y labels de entrenamiento.
    # -------- Español.
    data_train_es, data_test_es, label_train_es, label_test_es = train_test_split(
        vectorizacionLexicon, labels, random_state=1
    )
    #del df_es
    # -------- Ingles.
    #data_train_en, data_test_en, label_train_en, label_test_en = train_test_split(
    #    vectorizacionLexicon, labels, random_state=1
    #)
    #print("Division de datos terminada")
    #del df_en

    # ---------------- vectorizacion de los textos.
    # -------- Español.
    #training_data_es = vectorizer_es.fit_transform(data_train_es)
    #testing_data_es = vectorizer_es.transform(data_test_es)
    #del data_train_es, data_test_es
    #print("Vectorizacion en español terminada")
    # -------- Ingles.
    
   # ---------------- entrenamiento y guardado de los modelos.
    # -------- Español.
    ruta_modelo_es = './entrenamiento de modelos/modelos/' + str(type(modelo_es).__name__) + '_Sentiment_es.sav'
    modelo_es.fit(data_train_es, label_train_es)
    pickle.dump(modelo_es, open(ruta_modelo_es, 'wb'))
    print("Modelo de " + str(type(modelo_es).__name__) + " en español guardado en " + ruta_modelo_es)

    # -------- Ingles.
    #ruta_modelo_en = './entrenamiento de modelos/modelos/' + str(type(modelo_en).__name__) + '_Sentiment_en.sav'
    #modelo_en.fit(data_train_en, label_train_en)
    #pickle.dump(modelo_en, open(ruta_modelo_en, 'wb'))
    #print("Modelo de " + str(type(modelo_en).__name__) + " en ingles guardado en " + ruta_modelo_en)

    # ---------------- implementacion de los modelos.
    # -------- Español.
    predictions_es = modelo_es.predict(data_test_es)
    # -------- Ingles.
    #predictions_en = modelo_en.predict(data_test_en)
    print("Predicciones terminadas")
    
    
    # ---------------- Resultados de los modelos.
    # -------- Español.
    print("\nResultados " + str(type(modelo_es).__name__) + " Español:\n")
    print(classification_report(label_test_es, predictions_es))
    # -------- Ingles.
    #print("\nResultados " + str(type(modelo_en).__name__) + " Ingles:\n")
    #print(classification_report(label_test_en, predictions_en))


#### Entrenamiento de modelos NB:

In [27]:
entrenar_modelos_sentiments_supervisados('NB',vectorizacionLexicon,labels)

[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Separacion de datos por idioma terminada
Modelo de GaussianNB en español guardado en ./entrenamiento de modelos/modelos/GaussianNB_Sentiment_es.sav
Predicciones terminadas

Resultados GaussianNB Español:

              precision    recall  f1-score   support

           0       0.66      0.54      0.60       634
           1       0.58      0.72      0.64       675
           2       0.63      0.58      0.60       637

    accuracy                           0.62      1946
   macro avg       0.62      0.61      0.61      1946
weighted avg       0.62      0.62      0.61      1946



#### Entrenamiento de modelos DT:

In [28]:
entrenar_modelos_sentiments_supervisados('DT',vectorizacionLexicon,labels)


Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de DecisionTreeClassifier en español guardado en ./entrenamiento de modelos/modelos/DecisionTreeClassifier_Sentiment_es.sav
Predicciones terminadas

Resultados DecisionTreeClassifier Español:

              precision    recall  f1-score   support

           0       0.90      0.79      0.84       634
           1       0.99      0.93      0.96       675
           2       0.77      0.92      0.84       637

    accuracy                           0.88      1946
   macro avg       0.89      0.88      0.88      1946
weighted avg       0.89      0.88      0.88      1946



#### Entrenamiento de modelos RF:

In [29]:
entrenar_modelos_sentiments_supervisados('RF',vectorizacionLexicon,labels)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de RandomForestClassifier en español guardado en ./entrenamiento de modelos/modelos/RandomForestClassifier_Sentiment_es.sav
Predicciones terminadas

Resultados RandomForestClassifier Español:

              precision    recall  f1-score   support

           0       0.91      0.81      0.86       634
           1       0.96      0.93      0.95       675
           2       0.78      0.89      0.83       637

    accuracy                           0.88      1946
   macro avg       0.89      0.88      0.88      1946
weighted avg       0.89      0.88      0.88      1946



#### Entrenamiento de modelos GBT:

In [30]:
entrenar_modelos_sentiments_supervisados('GBT',vectorizacionLexicon,labels)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de AdaBoostClassifier en español guardado en ./entrenamiento de modelos/modelos/AdaBoostClassifier_Sentiment_es.sav
Predicciones terminadas

Resultados AdaBoostClassifier Español:

              precision    recall  f1-score   support

           0       0.88      0.80      0.84       634
           1       0.95      0.94      0.94       675
           2       0.79      0.86      0.82       637

    accuracy                           0.87      1946
   macro avg       0.87      0.87      0.87      1946
weighted avg       0.87      0.87      0.87      1946



#### Entrenamiento de modelos NN:

In [31]:
entrenar_modelos_sentiments_supervisados('NN',vectorizacionLexicon,labels)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de MLPClassifier en español guardado en ./entrenamiento de modelos/modelos/MLPClassifier_Sentiment_es.sav
Predicciones terminadas

Resultados MLPClassifier Español:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       634
           1       0.00      0.00      0.00       675
           2       0.33      1.00      0.49       637

    accuracy                           0.33      1946
   macro avg       0.11      0.33      0.16      1946
weighted avg       0.11      0.33      0.16      1946



/home/jose/Documentos/PytonVenv/venv_nlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
